Importing Dependencies

In [1]:
import pandas as pd

Loading health and population data

In [2]:
Health_file = pd.read_csv("HEALTH_STAT_17072021163319835.csv")
pop_file = pd.read_csv("HISTPOP_17072021165851274.csv")

health_df = pd.DataFrame(Health_file)
pop_df = pd.DataFrame(pop_file)

In [3]:
pop_df.head()

,LOCATION,Country,SEX,Sex,AGE,Age,TIME,Time,Value,Flag Codes,Flags
0,AUS,Australia,W,Women,TOTAL,Total,1964,1964,5516400.0,NaN,NaN
1,AUS,Australia,W,Women,TOTAL,Total,1965,1965,5626400.0,NaN,NaN
2,AUS,Australia,W,Women,TOTAL,Total,1966,1966,5757910.0,NaN,NaN
3,AUS,Australia,W,Women,TOTAL,Total,1967,1967,5859757.0,NaN,NaN
4,AUS,Australia,W,Women,TOTAL,Total,1968,1968,5965400.0,NaN,NaN


Cleaning Health Data

In [4]:
health_df["Variable"].values

array(['Females at birth', 'Females at birth', 'Females at birth', ...,
       'Males at age 80', 'Males at age 80', 'Males at age 80'],
      dtype=object)

In [5]:
pop_df_total = pop_df[pop_df["SEX"] == "T"]
pop_df_clean = pop_df_total.drop(["SEX", "Sex", "AGE", "Age", "TIME", "Flag Codes", "Flags"], axis=1)
population_df = pop_df_clean.rename(columns={"Time":"Year", "Value": "Population"})
population_df.head()

,LOCATION,Country,Year,Population
110,AUS,Australia,1964,11121600.0
111,AUS,Australia,1965,11340900.0
112,AUS,Australia,1966,11599498.0
113,AUS,Australia,1967,11799078.0
114,AUS,Australia,1968,12008635.0


In [6]:
health_df_total = health_df[health_df["Variable"] == "Total population at birth"]
health_df_total_cl = health_df_total.drop(["Variable", "VAR", "UNIT", "Measure", "YEA", "Flag Codes", "Flags"], axis=1)
health_stats = health_df_total_cl.rename(columns={"Value":"Life Expectancy at Birth"})
health_stats.head()

,COU,Country,Year,Life Expectancy at Birth
560,AUS,Australia,1964,70.7
561,AUS,Australia,1965,71.0
562,AUS,Australia,1966,70.8
563,AUS,Australia,1967,71.2
564,AUS,Australia,1968,70.9


Merging population and health stats to keep all of the population info 

In [7]:
pop_life = pd.merge(health_stats, population_df, how="right", on=["Country", "Year"])

In [8]:
pop_life = pop_life.drop("LOCATION", axis=1)

In [9]:
pop_life_cl = pop_life.drop_duplicates(subset=["Year","Country"])


Checking to see how many missing values there are

In [10]:
columns_2 = pop_life_cl.columns

for i in columns_2:
    print(pop_life_cl[i].isnull().sum())
    print(i)

810
COU
0
Country
0
Year
810
Life Expectancy at Birth
0
Population


In [11]:
#Exporting to excell in order to visualize it quicker
pop_life_cl.to_csv(r'health_pop_view.csv', index = False)

Replacing NaN values with Mean by country

In [87]:
pop_life_fill = pop_life_cl.fillna(pop_life_cl.groupby('Country').transform('mean'))
pop_life_fill = pop_life_cl.fillna(pop_life_cl.groupby('Year').transform('mean'))

In [88]:
pop_life_fill.head()

,COU,Country,Year,Life Expectancy at Birth,Population
0,AUS,Australia,1964,70.7,11121600.0
31,AUS,Australia,1965,71.0,11340900.0
62,AUS,Australia,1966,70.8,11599498.0
93,AUS,Australia,1967,71.2,11799078.0
124,AUS,Australia,1968,70.9,12008635.0


In [14]:
#Checking out NaN Values
columns = pop_life_fill.columns

for i in columns:
    print(pop_life_fill[i].isnull().sum())
    print(i)

810
COU
0
Country
0
Year
0
Life Expectancy at Birth
0
Population


In [15]:
pop_life_fill.drop('COU', axis=1)

,Country,Year,Life Expectancy at Birth,Population
0,Australia,1964,70.700000,1.112160e+07
31,Australia,1965,71.000000,1.134090e+07
62,Australia,1966,70.800000,1.159950e+07
93,Australia,1967,71.200000,1.179908e+07
124,Australia,1968,70.900000,1.200864e+07
...,...,...,...,...
95325,OECD - Total,2014,78.972727,1.272146e+09
95356,OECD - Total,2015,78.956818,1.279864e+09
95387,OECD - Total,2016,79.213636,1.287731e+09
95418,OECD - Total,2017,79.340909,1.294830e+09


In [89]:
pop_life_fill.shape[0]

3080

In [51]:
pop_life_fill.to_csv(r'health_pop.csv', index = False)

Loading medals count info

In [52]:
file_to_load_trey = pd.read_csv('complete_country_medals (7-29-2021).csv')

medal_count_df = pd.DataFrame(file_to_load_trey)

Replacing country names with standardized names

In [53]:
medal_count = medal_count_df['Country'].replace('Soviet Union', 'Russia', inplace=True)

In [54]:
medal_count = medal_count_df['Country'].replace('Russian Federation', 'Russia', inplace=True)

In [55]:
medal_count = medal_count_df['Country'].replace('Great Britain', 'England', inplace=True)
medal_count = medal_count_df['Country'].replace('United Kingdom', 'England', inplace=True)
medal_count = medal_count_df['Country'].replace('East Germany', 'Germany', inplace=True)
medal_count = medal_count_df['Country'].replace('West Germany', 'Germany', inplace=True)

In [56]:
medal_count = medal_count_df['Country'].replace("People's Republic of China", 'China', inplace=True)

Left Join (All from medal_count_df and only matching from stats)

In [90]:
medals_health_pop_stats = pd.merge(medal_count_df, pop_life_fill, how="left", on=["Country", "Year"])

In [91]:
medals_health_pop_stats.shape[0]

865

In [92]:
columns = medals_health_pop_stats.columns

for i in columns:
    print(medals_health_pop_stats[i].isnull().sum())
    print(i)

0
Country
0
Country Code
0
Gold
0
Silver
0
Bronze
0
Total
0
Year
0
Host_City
0
Host_Country
450
COU
385
Life Expectancy at Birth
385
Population


In [93]:
dropped_medals = medals_health_pop_stats.dropna(axis=0)
dropped_medals.shape[0]

415

In [112]:
# File to load into postgresDB
medals_health_pop_stats.to_csv(r'metal_count_stats.csv', index = False)

Right join verifies we only dropped non olympic years in the last file

In [95]:
medal_health_stats = pd.merge(medal_count_df, pop_life_fill, how="right", on=["Country", "Year"])

In [96]:
medal_health_stats['Host_City'] = medal_health_stats['Host_City'].fillna(0)

In [97]:
medal_health_stats.head()

,Country,Country Code,Gold,Silver,Bronze,Total,Year,Host_City,Host_Country,COU,Life Expectancy at Birth,Population
0,Australia,AUS,6.0,2.0,10.0,18.0,1964,Tokyo,JPN,AUS,70.7,11121600.0
1,Australia,NaN,NaN,NaN,NaN,NaN,1965,0,NaN,AUS,71.0,11340900.0
2,Australia,NaN,NaN,NaN,NaN,NaN,1966,0,NaN,AUS,70.8,11599498.0
3,Australia,NaN,NaN,NaN,NaN,NaN,1967,0,NaN,AUS,71.2,11799078.0
4,Australia,AUS,5.0,7.0,5.0,17.0,1968,Mexico City,MEX,AUS,70.9,12008635.0


In [98]:
medals_stats = medal_health_stats[medal_health_stats.Host_City != 0]

In [99]:
medals_stats.head()

,Country,Country Code,Gold,Silver,Bronze,Total,Year,Host_City,Host_Country,COU,Life Expectancy at Birth,Population
0,Australia,AUS,6.0,2.0,10.0,18.0,1964,Tokyo,JPN,AUS,70.7,11121600.0
4,Australia,AUS,5.0,7.0,5.0,17.0,1968,Mexico City,MEX,AUS,70.9,12008635.0
8,Australia,AUS,8.0,7.0,2.0,17.0,1972,Munich,DEU,AUS,72.0,13303664.0
12,Australia,AUS,0.0,1.0,4.0,5.0,1976,Montreal,CAN,AUS,72.8,14033083.0
16,Australia,AUS,2.0,2.0,5.0,9.0,1980,Moscow,RUS,AUS,74.6,14695356.0


In [100]:
medals_stats.shape[0]

480

In [101]:
columns = medals_stats.columns

for i in columns:
    print(medals_stats[i].isnull().sum())
    print(i)

0
Country
0
Country Code
0
Gold
0
Silver
0
Bronze
0
Total
0
Year
0
Host_City
0
Host_Country
65
COU
0
Life Expectancy at Birth
0
Population


Loading distance and GDP data

In [31]:
medals_health_pop_stats.columns

Index(['Country', 'Country Code', 'Gold', 'Silver', 'Bronze', 'Total', 'Year',
       'Host_City', 'Host_Country', 'COU', 'Life Expectancy at Birth',
       'Population'],
      dtype='object')

In [32]:
distance_csv = pd.read_csv("Country_Capitals_Distance.csv")
gdp_csv = ('GDP_Per_Year.csv')

In [33]:
disctance_df = pd.DataFrame(distance_csv)

In [34]:
disctance_df.head()

,Country,Capital,GDP (Billions),Latitude,Longitude,Latitude.1,Longitude.1,Latitude Hemisphere,Longitude Hemisphere,Distance From Tokyo 1964,...,Distance From Los Angeles 1984,Distance From Seoul 1988,Distance From Barcelona 1992,Distance From Atlanta 1996,Distance From Sydney 2000,Distance From Athens 2004,Distance From Beijing 2008,Distance From London 2012,Distance From Rio de Janeiro 2016,Distance From Tokyo 2020
0,Afghanistan,Kabul,19.29,34.28N,69.11E,34.28,69.11,North,East,3852.695234,...,10229.322310,3140.460940,3687.702265,8381.179784,6496.640378,2502.869092,2596.700595,3955.769518,7290.724741,3852.695234
1,Albania,Tirane,15.29,41.18N,19.49E,41.18,19.49,North,East,6571.702756,...,7536.133412,5848.844830,945.783004,5694.970611,8862.276379,329.679124,5281.563651,1277.354816,5591.135578,6571.702756
2,Algeria,Algiers,171.00,36.42N,03.08E,36.42,3.08,North,East,7456.896815,...,6626.090244,6739.011809,346.510726,4779.414037,9430.625168,1115.622926,6180.126784,1044.929886,4809.973012,7456.896815
3,American Samoa,Pago Pago,NaN,14.16S,170.43W,-14.16,-170.40,South,West,17274.499860,...,4379.072411,16595.202510,10172.101930,5742.998762,17612.491920,11170.603740,16081.216760,10341.615120,6972.944914,17274.499860
4,Andorra,Andorra la Vella,3.22,42.31N,01.32E,42.31,1.32,North,East,7566.659669,...,6552.808731,6843.529703,78.635860,4716.891500,9726.649424,1252.126668,6275.338342,628.914256,5113.942627,7566.659669


In [35]:
disctance_df.shape[0]

200

In [37]:
file_G = ("GDP_Per_Year.csv")
gdp_df = pd.read_csv(file_G)
gdp_df.head()

,LOCATION,Country,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,Australia,GDP,TOT,MLN_USD,A,1960,25034.740,NaN
1,AUS,Australia,GDP,TOT,MLN_USD,A,1961,25326.382,NaN
2,AUS,Australia,GDP,TOT,MLN_USD,A,1962,27913.206,NaN
3,AUS,Australia,GDP,TOT,MLN_USD,A,1963,30385.723,NaN
4,AUS,Australia,GDP,TOT,MLN_USD,A,1964,32694.467,NaN


In [38]:
#getting number of columns in gdp data
gdp_df.shape[0]

2478

In [102]:
# Standardizing column names
gdp_df = gdp_df.rename(columns={"TIME":"Year"})
gdp_df_drop = gdp_df.drop(["LOCATION", "INDICATOR", "SUBJECT", "FREQUENCY", "Flag Codes", "MEASURE"], axis=1)

In [103]:
gdp_distance = pd.merge(disctance_df, gdp_df_drop, how="right", on="Country")

In [104]:
gdp_distance.head()

,Country,Capital,GDP (Billions),Latitude,Longitude,Latitude.1,Longitude.1,Latitude Hemisphere,Longitude Hemisphere,Distance From Tokyo 1964,...,Distance From Barcelona 1992,Distance From Atlanta 1996,Distance From Sydney 2000,Distance From Athens 2004,Distance From Beijing 2008,Distance From London 2012,Distance From Rio de Janeiro 2016,Distance From Tokyo 2020,Year,Value
0,Australia,Canberra,1397.0,35.15S,149.08E,-35.15,149.0,South,East,4913.860856,...,9600.139656,13601.02612,149.534966,8495.174416,5456.561157,10092.5908,10526.41725,4913.860856,1960,25034.740
1,Australia,Canberra,1397.0,35.15S,149.08E,-35.15,149.0,South,East,4913.860856,...,9600.139656,13601.02612,149.534966,8495.174416,5456.561157,10092.5908,10526.41725,4913.860856,1961,25326.382
2,Australia,Canberra,1397.0,35.15S,149.08E,-35.15,149.0,South,East,4913.860856,...,9600.139656,13601.02612,149.534966,8495.174416,5456.561157,10092.5908,10526.41725,4913.860856,1962,27913.206
3,Australia,Canberra,1397.0,35.15S,149.08E,-35.15,149.0,South,East,4913.860856,...,9600.139656,13601.02612,149.534966,8495.174416,5456.561157,10092.5908,10526.41725,4913.860856,1963,30385.723
4,Australia,Canberra,1397.0,35.15S,149.08E,-35.15,149.0,South,East,4913.860856,...,9600.139656,13601.02612,149.534966,8495.174416,5456.561157,10092.5908,10526.41725,4913.860856,1964,32694.467


In [105]:
gdp_distance.shape[0]

2478

In [106]:
gdp_distance_drop = gdp_distance.drop(["Latitude", "Longitude", "Latitude.1", "Longitude.1"], axis=1)

In [107]:
gdp_distance_drop.head()

,Country,Capital,GDP (Billions),Latitude Hemisphere,Longitude Hemisphere,Distance From Tokyo 1964,Distance From Mexico City 1968,Distance From Munich 1972,Distance From Montreal 1976,Distance From Moscow 1980,...,Distance From Barcelona 1992,Distance From Atlanta 1996,Distance From Sydney 2000,Distance From Athens 2004,Distance From Beijing 2008,Distance From London 2012,Distance From Rio de Janeiro 2016,Distance From Tokyo 2020,Year,Value
0,Australia,Canberra,1397.0,South,East,4913.860856,14055.77408,9451.196419,13465.23217,8731.68456,...,9600.139656,13601.02612,149.534966,8495.174416,5456.561157,10092.5908,10526.41725,4913.860856,1960,25034.740
1,Australia,Canberra,1397.0,South,East,4913.860856,14055.77408,9451.196419,13465.23217,8731.68456,...,9600.139656,13601.02612,149.534966,8495.174416,5456.561157,10092.5908,10526.41725,4913.860856,1961,25326.382
2,Australia,Canberra,1397.0,South,East,4913.860856,14055.77408,9451.196419,13465.23217,8731.68456,...,9600.139656,13601.02612,149.534966,8495.174416,5456.561157,10092.5908,10526.41725,4913.860856,1962,27913.206
3,Australia,Canberra,1397.0,South,East,4913.860856,14055.77408,9451.196419,13465.23217,8731.68456,...,9600.139656,13601.02612,149.534966,8495.174416,5456.561157,10092.5908,10526.41725,4913.860856,1963,30385.723
4,Australia,Canberra,1397.0,South,East,4913.860856,14055.77408,9451.196419,13465.23217,8731.68456,...,9600.139656,13601.02612,149.534966,8495.174416,5456.561157,10092.5908,10526.41725,4913.860856,1964,32694.467


In [108]:
gdp_distance_drop['Year'].dtype

dtype('int64')

In [109]:
gdp_distance_drop.dropna(axis=0)

,Country,Capital,GDP (Billions),Latitude Hemisphere,Longitude Hemisphere,Distance From Tokyo 1964,Distance From Mexico City 1968,Distance From Munich 1972,Distance From Montreal 1976,Distance From Moscow 1980,...,Distance From Barcelona 1992,Distance From Atlanta 1996,Distance From Sydney 2000,Distance From Athens 2004,Distance From Beijing 2008,Distance From London 2012,Distance From Rio de Janeiro 2016,Distance From Tokyo 2020,Year,Value
0,Australia,Canberra,1397.00,South,East,4913.860856,14055.774080,9451.196419,13465.232170,8731.684560,...,9600.139656,13601.026120,149.534966,8495.174416,5456.561157,10092.590800,10526.417250,4913.860856,1960,25034.740
1,Australia,Canberra,1397.00,South,East,4913.860856,14055.774080,9451.196419,13465.232170,8731.684560,...,9600.139656,13601.026120,149.534966,8495.174416,5456.561157,10092.590800,10526.417250,4913.860856,1961,25326.382
2,Australia,Canberra,1397.00,South,East,4913.860856,14055.774080,9451.196419,13465.232170,8731.684560,...,9600.139656,13601.026120,149.534966,8495.174416,5456.561157,10092.590800,10526.417250,4913.860856,1962,27913.206
3,Australia,Canberra,1397.00,South,East,4913.860856,14055.774080,9451.196419,13465.232170,8731.684560,...,9600.139656,13601.026120,149.534966,8495.174416,5456.561157,10092.590800,10526.417250,4913.860856,1963,30385.723
4,Australia,Canberra,1397.00,South,East,4913.860856,14055.774080,9451.196419,13465.232170,8731.684560,...,9600.139656,13601.026120,149.534966,8495.174416,5456.561157,10092.590800,10526.417250,4913.860856,1964,32694.467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2432,Madagascar,Antananarivo,14.19,South,East,6278.612532,8407.647396,4998.065779,8034.024336,5134.877977,...,4814.541306,8045.533417,5770.586311,4086.030776,5497.214207,5473.171039,4949.359533,6278.612532,2014,37125.674
2433,Madagascar,Antananarivo,14.19,South,East,6278.612532,8407.647396,4998.065779,8034.024336,5134.877977,...,4814.541306,8045.533417,5770.586311,4086.030776,5497.214207,5473.171039,4949.359533,6278.612532,2015,37860.909
2434,Madagascar,Antananarivo,14.19,South,East,6278.612532,8407.647396,4998.065779,8034.024336,5134.877977,...,4814.541306,8045.533417,5770.586311,4086.030776,5497.214207,5473.171039,4949.359533,6278.612532,2016,40437.411
2435,Madagascar,Antananarivo,14.19,South,East,6278.612532,8407.647396,4998.065779,8034.024336,5134.877977,...,4814.541306,8045.533417,5770.586311,4086.030776,5497.214207,5473.171039,4949.359533,6278.612532,2017,40952.508


In [111]:
# File to load into postgresDB
gdp_distance_drop.to_csv(r'gdp_distance.csv', index = False)

In [4]:
import pandas as pd

In [50]:
file = 'C:/Users/tucke/OneDrive/Desktop/Data Analysis Bootcamp/Olympics/HH/Aug2/Competition_Cleaned.csv'

masterReseultData = pd.read_csv(file)

masterReseultData_df = pd.DataFrame(masterReseultData)

In [87]:
file_2 = 'C:/Users/tucke/OneDrive/Desktop/Data Analysis Bootcamp/Olympics/Trey_Graham/Reviews_Edits/Use for analysis/country_codes_table (version 1).xlsb.csv'

noc = pd.read_csv(file_2)

noc_df = pd.DataFrame(noc)

In [88]:
noc = noc.rename(columns={'ISO 3166 ALPHA-3':'NOC'})
noc.head()

,NOC,NOC.1,Country
0,AFG,AFG,Afghanistan
1,AHO,AHO,Netherlands Antilles
2,ALB,ALB,Albania
3,ALG,ALG,Algeria
4,AND,AND,Andorra


In [89]:
masterReseultData_df.head()

,Event,NOC,Pos.,Year,Discipline,Gender Cleaned,Event Cleaned
0,"Balance Beam, Women",TCH,1,1964,Artistic Gymnastics,Women,Balance Beam
1,"Balance Beam, Women",URS,2,1964,Artistic Gymnastics,Women,Balance Beam
2,"Balance Beam, Women",URS,3,1964,Artistic Gymnastics,Women,Balance Beam
3,"Floor Exercise, Men",ITA,1,1964,Artistic Gymnastics,Men,Floor Exercise
4,"Floor Exercise, Men",URS,2,1964,Artistic Gymnastics,Men,Floor Exercise


In [90]:
master_df = masterReseultData_df.drop('Event', axis=1)
master_df = master_df.rename(columns={'Pos.':'Place'})
master_df.head()

,NOC,Place,Year,Discipline,Gender Cleaned,Event Cleaned
0,TCH,1,1964,Artistic Gymnastics,Women,Balance Beam
1,URS,2,1964,Artistic Gymnastics,Women,Balance Beam
2,URS,3,1964,Artistic Gymnastics,Women,Balance Beam
3,ITA,1,1964,Artistic Gymnastics,Men,Floor Exercise
4,URS,2,1964,Artistic Gymnastics,Men,Floor Exercise


In [91]:
master_df.Place.unique()

array([1, 2, 3], dtype=int64)

In [92]:
master_df['Place'] = master_df['Place'].map({1:'Gold', 2:'Silver', 3:'Bronze'}, na_action=None)
master_df.head()

,NOC,Place,Year,Discipline,Gender Cleaned,Event Cleaned
0,TCH,Gold,1964,Artistic Gymnastics,Women,Balance Beam
1,URS,Silver,1964,Artistic Gymnastics,Women,Balance Beam
2,URS,Bronze,1964,Artistic Gymnastics,Women,Balance Beam
3,ITA,Gold,1964,Artistic Gymnastics,Men,Floor Exercise
4,URS,Silver,1964,Artistic Gymnastics,Men,Floor Exercise


In [101]:
master = pd.merge(noc, master_df, how='right', on=['NOC'])
master = master.dropna(axis=0)
master = master.drop('NOC.1', axis=1)
master.head()

,NOC,Country,Place,Year,Discipline,Gender Cleaned,Event Cleaned
0,TCH,Czechoslovakia,Gold,1964,Artistic Gymnastics,Women,Balance Beam
1,TCH,Czechoslovakia,Gold,1964,Artistic Gymnastics,Women,Horse Vault
2,TCH,Czechoslovakia,Gold,1964,Artistic Gymnastics,Women,Individual All-Around
3,TCH,Czechoslovakia,Silver,1964,Artistic Gymnastics,Women,Team All-Around
4,TCH,Czechoslovakia,Silver,1964,Athletics,Men,"1,500 metres"


In [97]:
master['Country'].isnull().sum()

0

In [98]:
master.shape[0]

10520